## Elementos finitos linear 1D

## Montar a matrix de massa

In [100]:
using Jacobi

## Funções

In [101]:
remap(x,a,b) = ((1-x)*a + (1+x)*b)/2

remap (generic function with 1 method)

In [102]:
function ψj(p,E,Q)
    if(p == 1)
     return  (1-E)/2
        elseif(p == 2)
     return  (1+E)/2
    else
     return  (1-E)*(1+E)/4 .* jacobi(E, p-3, 1, 1)
    end
end 

ψj (generic function with 1 method)

In [103]:
function  ϕ_matrix(ψj,Q,M)
    ϕ = zeros(Q,M)
    ξ = zglj(Q)
    for i in 1:M
        for j in 1:Q
            ϕ[j,i] = ψj(i,ξ[j],M)
        end
    end
  return ϕ
end

ϕ_matrix (generic function with 1 method)

In [104]:
function  ϕ_matrix_interp(ψj,Q,M)
    ϕ = zeros(Q,M)
    ξ = linspace(-1,1,Q)
    for i in 1:M
        for j in 1:Q
            ϕ[j,i] = ψj(i,ξ[j],M)
        end
    end
  return ϕ
end

ϕ_matrix_interp (generic function with 1 method)

In [105]:
function Me_matrix(ϕ,w,M,Q,a,b)
    L = zeros(M,M)
    jac = (b-a)/2
    for i in 1:M
        for j in 1:M
           m= 0.0
            for q in 1:Q
                m = m + ϕ[q,i]*ϕ[q,j]*w[q]*jac
            end
            L[i,j] = m
        end
    end
    return L
end

Me_matrix (generic function with 1 method)

### $ϕ$ e  matriz de massa

In [106]:
Nel = 2
I = 3
nb = Nel + 1
Q = 10

#indices locais
ib = 1:2
ii = [3:I]

#limites
a = -1; 
b = 1;

z = zglj(Q)
w = wglj(z)
ϕ = ϕ_matrix(ψj,Q,I);
xn = [linspace(a,b,Nel+1);];

In [107]:

dof_map = zeros(Int, 2, Nel)
for i = 1:Nel
    dof_map[1,i] = i
    dof_map[2,i] = i+1
end
dof_map

2x2 Array{Int64,2}:
 1  2
 2  3

In [108]:
using ArrayViews
using Base.LinAlg.BLAS.gemm!
using Base.LinAlg.BLAS.gemv!
using Base.LinAlg.LAPACK.potrf!
using Base.LinAlg.LAPACK.potrs!

In [109]:
ib = 1:2
ii = [3]

Abb = zeros(Nel+1,Nel+1)

for e in 1:Nel
    Me = Me_matrix(ϕ, w, I, Q, xn[e], xn[e+1]) ;
    Mbb = Me[ib,ib]
    Mbi = Me[ib,ii]
    Mib = Me[ii,ib]
    Mii = Me[ii,ii]
    
    potrf!('L',Mii); #Mii^-1
    M = copy(Mib)
    potrs!('L',Mii,M) # M = Mii^-1 * Mbi
    gemm!('T', 'N', -1.0, Mib, M, 1.0, Mbb) # Mbb = -1*M*Mib + Mbb
    
    for i in 1:2
        ig = dof_map[e,i]
        for j in 1:2
            jg = dof_map[e,j]
            Abb[ig,jg] = Mbb[i,j]
        end
    end
end
potrf!('L',Abb)
Abb

3x3 Array{Float64,2}:
  0.353553  -0.0416667   0.0      
 -0.117851   0.333333   -0.0416667
  0.0       -0.125       0.330719 

## RHS

In [110]:
fun(x) = cos(2*pi*x)
#calcula F

F = zeros(Nel+1 + Nel*(I-2))
for e = 1:Nel
    Me = Me_matrix(ϕ, w, I, Q, xn[e], xn[e+1]) ;
    Fe = Me * fun(linspace(xn[e],xn[e+1],I))
    for i = 1:2
        ig = dof_map[i,e]
        F[ig] += Fe[i]
    end
    for i = 1:I-2
        ig = nb +(e -1)*(I-2) + i
        F[ig] += Fe[i]
    end
end
F

5-element Array{Float64,1}:
  0.25     
  0.166667 
 -0.0833333
  0.25     
  0.25     

In [111]:
for e in 1:Nel
    Me = Me_matrix(ϕ, w, I, Q, xn[e], xn[e+1]) ;
    Mbb = Me[ib,ib]
    Mbi = Me[ib,ii]
    Mib = Me[ii,ib]
    Mii = Me[ii,ii]
    fb  = F[dof_map[e,:]]
    i_g = nb + (e -1)*(I-2) + collect(1:I-2)
    fi  = F[i_g]
      
    potrf!('L',Mii);
    M = copy(Mib)
    potrs!('L',Mii,M) # M = Mii^-1 * M
    gemv!('T',-1.0,M,vec(fi), 1.0, vec(fb)) #fbb = -1*M^T*fi + fb
    for i in 1:2
        ig = dof_map[e,i]
        F[ig] = fb[i]
    end
    
end


# Solve $A^*_{bb}\ u_b =\ f^*_b$

In [112]:
U = zeros(Nel+1+Nel*(I-2))
Ub = sub(U,1:Nel+1)
Ui = sub(U,(Nel+2):length(U))
Ub = Abb\Fb
U[1:Nel+1] = Ub;

### $u_i = A_{ii}^{-1}f_i - A_{ii}^{-1} A_{bi}^t u_b$

In [113]:

for e in 1:Nel
    Me = Me_matrix(ϕ, w, I, Q, xn[e], xn[e+1]) ;
    Mbb = Me[ib,ib]
    Mbi = Me[ib,ii]
    Mib = Me[ii,ib]
    Mii = Me[ii,ii]
    
    ub  = U[dof_map[e,:]]
    i_g = nb + (e -1)*(I-2) + collect(1:I-2)
    fi  = F[i_g]

    potrf!('L',Mii);
    M = copy(Mbi)
   
    gemv!('T',-1.0,M,vec(ub), 1.0, vec(fi)) #fi = -1*M^T*ub + fi
   
    potrs!('L',Mii,fi) #fi = Mii^-1 * fi
    
    for i in 1:I-2
        ig = nb + (e -1)*(I-2) + i 
        U[ig] = fi[i]
    end
    
end

In [114]:
U

5-element Array{Float64,1}:
 -3.30074e-16
 -5.96404e-16
 -1.63703e-16
  7.5        
  7.5        

In [115]:
fun(linspace(-1,1,5))

5-element Array{Float64,1}:
  1.0
 -1.0
  1.0
 -1.0
  1.0

In [116]:
[-1:.5:1]

5-element Array{Float64,1}:
 -1.0
 -0.5
  0.0
  0.5
  1.0